# Question 2

# Group Details

<table width = 100%>
<tr>
<th>ID</th>
<th>Student Name</th>
<th>Student Email</th>
</tr>
<tr>
<td>1181200713</td>
<td>Jianxi Kita</td>
<td>1181200713@student.mmu.edu.my</td>
</tr>
<tr>
<td>1181200774</td>
<td>Bryan Bong Zi Hao</td>
<td>1181200774@student.mmu.edu.my</td>
</tr>
<tr>
<td>1181201475</td>
<td>Muhammad Nabil Anwar bin Md Zaid</td>
<td>1181201475@student.mmu.edu.my</td>
</tr>

</table>

*If the program is unable to boot up or takes a long time. Try restarting the kernel or checking if any other programs are consuming computing resource

In [ ]:
%pip install pyamaze

# 1. Breadth-first Search

In [ ]:
from collections import deque
import time

def BFS(m, goal):
    # Initialization
    start = (m.rows, m.cols)
    explored = set([start]) # utilizing set for better performance compared to list 
    frontier = deque([start]) 

    # print("original frontier=", frontier)
    # print("original explored=", explored)

    bfsPath = {}
    searchPath = []
    visitedCount = {}  # Counter to track visited cells

    start_time = time.time()

    # Continue loop as long as frontier is not empty
    while frontier:
        # Pops based on FIFO
        currCell = frontier.popleft()
        searchPath.append(currCell)

        # If current cell == goal, then end
        if currCell == (goal[0], goal[1]):
            break

        # Look into all 4 directions
        for d in "ESNW":
            if m.maze_map[currCell][d]:
                if d == "E":
                    childCell = (currCell[0], currCell[1] + 1)
                elif d == "W":
                    childCell = (currCell[0], currCell[1] - 1)
                elif d == "S":
                    childCell = (currCell[0] + 1, currCell[1])
                elif d == "N":
                    childCell = (currCell[0] - 1, currCell[1])

                if childCell in explored:
                    # Check if the cell has been visited less than 3 times
                    if visitedCount.get(childCell, 0) < 3:
                        visitedCount[childCell] = visitedCount.get(childCell, 0) + 1
                        frontier.append(childCell)
                    continue

                explored.add(childCell)
                frontier.append(childCell)

                # print("frontier=", frontier)
                # print("explored=", explored)

                bfsPath[childCell] = currCell

    end_time = time.time()
    elapsed_time = end_time - start_time    

    fwdPath = {}
    cell = (goal[0], goal[1])

    while cell != start:
        fwdPath[bfsPath[cell]] = cell
        cell = bfsPath[cell]

    return searchPath,bfsPath,fwdPath, elapsed_time


# 2. A* Search

In [ ]:
from pyamaze import maze,agent,COLOR,textLabel
from queue import PriorityQueue
import time

def h(cell1,cell2):
    x1,y1=cell1
    x2,y2=cell2
    return abs(x1-x2) + abs(y1-y2) 

def aStar(m,goal):
    start=(m.rows,m.cols)
    g_score={cell:float('inf') for cell in m.grid}
    g_score[start]=0
    f_score={cell:float('inf') for cell in m.grid}
    f_score[start]=h(start,(goal[0],goal[1]))

    open_set=PriorityQueue()
    open_set.put((h(start,(goal[0],goal[1])),h(start,(goal[0],goal[1])),start))
    aPath={}
    
    searchPath = []
    start_time = time.time()
    while not open_set.empty():
        currCell = open_set.get()[2]
        searchPath.append(currCell)
        
        if currCell == (goal[0],goal[1]):
            break
        for d in 'ESNW':
            if m.maze_map[currCell][d] == True:
                if d == 'E':
                    childCell = (currCell[0],currCell[1]+1)
                if d == 'S':
                    childCell = (currCell[0]+1,currCell[1])
                if d == 'N':
                    childCell = (currCell[0]-1,currCell[1])
                if d == 'W':
                    childCell = (currCell[0],currCell[1]-1)
                    
                temp_g_score = g_score[currCell] + 1
                temp_f_score = temp_g_score+h(childCell,(goal[0],goal[1]))
                
                if temp_f_score < f_score[childCell]:
                    g_score[childCell] = temp_g_score
                    f_score[childCell] = temp_f_score
                    open_set.put((temp_f_score,h(childCell,(goal[0],goal[1])),childCell))
                    aPath[childCell] = currCell    
    fwdPath={}
    cell=(goal[0],goal[1])
    
    end_time = time.time()
    elapsed_time = end_time - start_time

    while cell!=start:
        fwdPath[aPath[cell]]=cell
        cell=aPath[cell]    
    
    return fwdPath,aPath,searchPath,elapsed_time

# 3. Outputting the Path to a text file

In [ ]:

import os
def FileExists(file_path):
	if os.path.exists(file_path):
		return True
	else:
		return False

def exportOutput(filename,searchName,pathName,elapsed_time = 0,color = "",envText = ""):
    text = "\n" + searchName + " Search Path\n"
    text += "Search Path Colour: " + color + "\n"
    text += "Elapsed Time: " + str(elapsed_time) + "\n"
    text +=  envText + "\n"
    text += "\n=============================================\n"
    text += "{" + "\n".join("{!r}: {!r},".format(k, v) for k, v in pathName.items()) + "}"
    i = 1
    # get directory from start of path to last slash
    directory = filename[:filename.rfind('/')]
    while (FileExists(filename)) :
        filename = filename[:-4]
        filename = directory + "/"
        filename += searchName +" Output" + str(i) +  ".txt"
        i+= 1
    with open(filename,'w') as file:
        file.write(text)

# Main Driver Functions

## Maze creation

In [ ]:
import random
from pyamaze import maze, agent, COLOR

a = 10 #size of maze
b = 10 #size of maze
c = 5	 #loop percent

# get random value for goal 
goal = random.randint(1,a - 1),random.randint(1,b - 1)
# goal = a,b #uncomment if want to set to specific goal and comment the line above
envText = "Maze Size: " + str(a) + "x" + str(b) + "\nLoop Percent: " + str(c) + "%" + "\nGoal: " + str(goal) 

m = maze(a,b) #instantiation of maze
m.CreateMaze(goal[0],goal[1],loopPercent=c,saveMaze=True) #saveMaze exports the maze into a csv file

# BFS Search
bfsPath=BFS(m,goal)
bfsSearchAgent=agent(m,footprints=True,color=COLOR.yellow,filled=True)
bfsPathAgent=agent(m,footprints=True)
m.tracePath({bfsSearchAgent:bfsPath[0]},showMarked=True)
m.tracePath({bfsPathAgent:bfsPath[2]})
exportOutput("output/bfs/BFS Output.txt","BFS",bfsPath[1],bfsPath[3],"Yellow", envText)
# A* Search
aPath=aStar(m,goal) 
aSearchAgent=agent(m,footprints=True,color=COLOR.red,filled=True)
aPathAgent=agent(m,footprints=True)
m.tracePath({aSearchAgent:aPath[2]},showMarked=True)
m.tracePath({aPathAgent:aPath[0]})
exportOutput("output/astar/AStar Search Output.txt","AStar Search",aPath[1],aPath[3],"Red", envText)

m.run()

# Loading an existing maze

In [ ]:
from pyamaze import maze, agent, COLOR

mazePath = 'maze10x10.csv'
# mazePath = 'maze15x15.csv'
# mazePath = 'maze20x20.csv'
# mazePath = 'maze25x25.csv'
# mazePath = 'maze30x30.csv'
# mazePath = 'maze35x35.csv'

# get maze size from file name
a = int(mazePath[4:6])
b = int(mazePath[7:9])

m = maze() #instantiation of maze
goal = 5,5 #modify this to set custom goal
envText = "Maze Size: " + str(a) + "x" + str(b)

m.CreateMaze(goal[0],goal[1],loadMaze=mazePath) #saveMaze exports the maze into a csv file

# BFS Search
bfsPath=BFS(m,goal)
bfsSearchAgent=agent(m,footprints=True,color=COLOR.yellow,filled=True)
bfsPathAgent=agent(m,footprints=True)
m.tracePath({bfsSearchAgent:bfsPath[0]},showMarked=True)
m.tracePath({bfsPathAgent:bfsPath[2]}) 
exportOutput("output/bfs/BFS Output.txt","BFS",bfsPath[1],bfsPath[3],"Yellow", envText)

# A* Search
aPath=aStar(m,goal) 
aSearchAgent=agent(m,footprints=True,color=COLOR.red,filled=True)
aPathAgent=agent(m,footprints=True)
m.tracePath({aSearchAgent:aPath[2]},showMarked=True)
m.tracePath({aPathAgent:aPath[0]})
exportOutput("output/astar/AStar Search Output.txt","AStar Search",aPath[1],aPath[3],"Red", envText)

m.run()

# Summary

After evaluating the outputs from the five different mazes, it can be concluded that A* search outperformed BFS (Breadth-First Search) in terms of speed. A* search utilizes heuristic information and efficiently explored the maze and found the optimal path. On the other hand, BFS systematically explored all neighboring nodes, resulting in longer search times compared to A*. Despite the varying maze sizes and loop percentages, A* consistently exhibited faster search times, showcasing its ability to prioritize more promising paths. In a nutshell, A* search proves to be a more time-efficient algorithm for maze navigation compared to BFS in these scenarios.